In [1]:
import multiprocessing as mp
import os
import linecache
from functools import partial
print("Procesadores: {}".format(mp.cpu_count()))

Procesadores: 2


Montar Drive, copiar archivo a la maquina local de colaboratory

In [2]:
!cp /content/drive/MyDrive/BigData/checkouts_seattle.zip .

cp: cannot stat '/content/drive/MyDrive/BigData/checkouts_seattle.zip': No such file or directory


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Descomprimir

In [6]:
!unzip drive/MyDrive/BigData/checkouts_seattle.zip

Archive:  drive/MyDrive/BigData/checkouts_seattle.zip
  inflating: Checkouts by Title FAQs.pdf  
  inflating: checkouts-by-title.csv  
  inflating: socrata_metadata.json   


Lista de archivos con detalle (El h en -lh significa "human", para leer el tamaño de los archivos en KB, MB, GB... y no en bytes)

In [8]:
ls -lh 

total 7.2G
-rw-r--r-- 1 root root 7.2G Dec  3  2019  checkouts-by-title.csv
-rw-r--r-- 1 root root 363K Dec  3  2019 'Checkouts by Title FAQs.pdf'
drwx------ 6 root root 4.0K Apr  1 19:14  drive/
drwxr-xr-x 1 root root 4.0K Mar 23 14:22  sample_data/
-rw-r--r-- 1 root root  16K Dec  3  2019  socrata_metadata.json


Encabezado de archivo

In [9]:
!tail checkouts-by-title.csv

Physical,Horizon,BOOK,2014,8,6,Fueled for adventure / adapted by Christine Peymani and Adam B. Murr ; illustrated by Carson Van Osten.,"Peymani, Christine","Automobiles Juvenile fiction, Automobile racing Juvenile fiction, Movie novels","Golden Books,",c2011.
Digital,OverDrive,EBOOK,2014,8,1,Why God Won't Go Away: Brain Science and the Biology of Belief,"Andrew Newberg, M.D.","Nonfiction, Religion & Spirituality, Science","Random House, Inc.",2012
Digital,OverDrive,EBOOK,2014,8,2,Criss Cross,Lynne Rae Perkins,"Historical Fiction, Young Adult Fiction",HarperCollins Publishers Inc.,2010
Physical,Horizon,BOOK,2014,8,1,The woman who shot Mussolini / Frances Stonor Saunders.,"Saunders, Frances Stonor","Gibson Violet, Mussolini Benito 1883 1945 Assassination attempt 1926 April 7, Assassins Biography, Catholic converts Biography, Aristocracy Social class Ireland Dublin Biography, Ashbourne Edward Gibson Baron 1837 1913 Family, Ireland History 1837 1901 Biography, Dublin Ireland Biography, Mus

Por fines didacticos, solo se revisan los primeros 10M de registros

In [10]:
#subsample of 10M of file
!head -n 1000000 checkouts-by-title.csv > checkouts10M.csv

Contamos las líneas del archivo original, son ~34M

Comandos
wc <- word count, para hacer conteo en general de caracteres, el argumento -l hace que cuente las líneas

In [11]:
%%time
!wc -l checkouts-by-title.csv

34892624 checkouts-by-title.csv
CPU times: user 152 ms, sys: 31.2 ms, total: 183 ms
Wall time: 29.4 s


Verificamos que el archivo muestra efectivamente tiene 10M

In [12]:
%%time
!wc -l checkouts10M.csv

1000000 checkouts10M.csv
CPU times: user 10.8 ms, sys: 4.42 ms, total: 15.3 ms
Wall time: 817 ms


Setteamos el nombre del archivo y el número de lineas a procesar

In [13]:
filename = "checkouts10M.csv"
lines = 1000000


#En Paralelo

In [28]:
#threads = mp.cpu_count() #threads disponibles en la máquina
threads = 4
num_lines = lines
split_size = num_lines // threads if num_lines // threads > 0 else 1 #Tamaños a procesar por cada procesador
parts = [x for x in range(0, num_lines, split_size)] #Donde inicia 
parts

[0, 250000, 500000, 750000]

In [29]:
def countWordsInTitles(filename, chunksize, start):
  wordCount = {}
  counter = 0
  print("Calling from {}".format(start))
  for current_line in range(start, (start+chunksize)+1):
    ln = linecache.getline(filename, current_line)
    try:
      words = ln.split(",")[6].split(" ")
      #print(words)
      for w in words:
        if w.upper() not in wordCount.keys():
          wordCount[w.upper()] = 0
        wordCount[w.upper()] += 1
    except Exception as e:
      print("null title {} ".format(str(e)))      
    counter += 1
    if counter % 1000000 == 0:
      print("From {start} in line {counter}".format(start=start, counter=counter))
  return wordCount

In [30]:
%%time
pool = mp.Pool(processes=threads)
func = partial(countWordsInTitles, filename, split_size)
res = pool.map(func, parts)
pool.close()
pool.join()
# "res" entonces es una lista de diccionarios con las cuentas de palabras por cada worker.

Calling from 0
Calling from 500000
Calling from 250000
Calling from 750000
null title list index out of range 
CPU times: user 183 ms, sys: 132 ms, total: 315 ms
Wall time: 7.42 s


## De forma secuencial

In [31]:
%%time
wordCount = {}
counter = 0
#f1 = open(filename, 'r')
for i in range(1,lines):
  ln = linecache.getline(filename, i)
  try:    
    words = ln.split(',')[6].split(" ")  
    for w in words:
      if w.upper() not in wordCount.keys():
        wordCount[w.upper()] = 0
      wordCount[w.upper()] += 1
  except Exception as e:
    print("null title {} ".format(str(e)))      
  counter += 1
  if counter % 1000000 == 0:
    print("From {start} in line {counter}".format(start=0, counter=counter))
  


CPU times: user 8.87 s, sys: 63.6 ms, total: 8.94 s
Wall time: 8.95 s


In [ ]:
#Este es el sweetspot, se puede ver que secuencial tarda un poco mas, "aprox 10% en este caso"

In [18]:
res[0]

{'TITLE': 72,
 "MCGRAW-HILL'S": 11,
 'DICTIONARY': 830,
 'OF': 105321,
 'AMERICAN': 6294,
 'SLANG': 50,
 'AND': 85054,
 'COLLOQUIAL': 24,
 'EXPRESSIONS': 37,
 '/': 171853,
 'RICHARD': 2767,
 'A.': 2154,
 'SPEARS.': 25,
 '"EMMA': 3,
 'RED': 1728,
 'MIDNIGHT': 555,
 'JUST': 1020,
 'THE': 176736,
 'FINANCIAL': 450,
 'FACTS': 307,
 'HOW': 10242,
 'TO': 44359,
 'IDENTIFY': 62,
 'NUGGETS': 12,
 'USABLE': 14,
 'NEWS': 294,
 'MINIMIZE': 11,
 'INFORMATION': 267,
 'OVERLOAD': 12,
 'SINGLE': 210,
 'SHARD': 13,
 'PAUL': 2303,
 "KIRK'S": 3,
 'CHAMPIONSHIP': 35,
 'BARBECUE': 62,
 ':': 76332,
 'BBQ': 22,
 'YOUR': 12861,
 'WAY': 2802,
 'GREATNESS': 51,
 'WITH': 19255,
 '575': 2,
 "LIP-SMACKIN'": 2,
 'RECIPES': 2573,
 'FROM': 16645,
 'BARON': 61,
 'KIRK': 103,
 'BOB': 1090,
 'LYON.': 32,
 'LIVE': 2351,
 'THIS': 1660,
 'IS': 5136,
 'HOUSE': 3116,
 '"BANDIDO': 1,
 'OSCAR': 223,
 '""ZETA""': 1,
 'ACOSTA': 1,
 'CHICANO': 8,
 'EXPERIENCE': 433,
 'ILAN': 19,
 'STAVANS."': 1,
 'COLUMBIA': 610,
 'RIVER': 995,


In [19]:
#Eso fue el mapper, este es el reduce
#cuantos mappers teniamos?
len(res)

2

In [20]:
reduced = {} #Creamos el diccionario reducido
for i in res: #Para cada elemento del mapper
  for key in i.keys(): #Recorremos las llaves
    if key not in reduced.keys(): #si no existe la creamos con el primer valor que encuentra para esa llave
      reduced[key] = i[key] 
    else:
      reduced[key] = reduced[key] + i[key] #si existe, entonces le suma el valor

In [21]:
print("Total keys in first", len(res[0].keys()))
print("Total keys in second", len(res[1].keys()))

Total keys in first 152760
Total keys in second 194153


In [22]:
print("Total Keys reduced", len(reduced.keys()))

Total Keys reduced 240508


In [23]:
reduced.keys() #llaves, tomar ejemplo aleatorio

Output hidden; open in https://colab.research.google.com to view.

In [24]:
reduced["AMERICAN"]

11719

In [25]:
res[0]["AMERICAN"]

6294

In [26]:
res[1]["AMERICAN"]

5425

In [27]:
res[0]["AMERICAN"] + res[1]["AMERICAN"]

11719